#  GPT Adventure
*By Nathan Whitmore, based on a tutorial by [Max Woolf](http://minimaxir.com)*

This is the game! Because it takes a fairly large amount of computing power and data storage, it runs on Google's servers through their Colab service. All you need to do to play it is


*  File -> Save to save the file on your Google Drive
*   Run the "Setup steps" code
*  Run the "Play the game" code





##Setup steps
Before playing this must be run to put the needed files on Google's servers. Click the arrow on the top left of the box below to run setup.

In [ ]:
#Click the number or arrow to the left to run

!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
!wget -O checkpoint_run1.tar https://northwestern.box.com/shared/static/8k34b5sfq1ib5e4kmwjehxtpjotfdszl.tar
!tar -xvf checkpoint_run1.tar
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)
print("Setup complete!")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

--2019-12-20 23:38:32--  https://northwestern.box.com/shared/static/8k34b5sfq1ib5e4kmwjehxtpjotfdszl.tar
Resolving northwestern.box.com (northwestern.box.com)... 185.235.236.197
Connecting to northwestern.box.com (northwestern.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/8k34b5sfq1ib5e4kmwjehxtpjotfdszl.tar [following]
--2019-12-20 23:38:33--  https://northwestern.box.com/public/static/8k34b5sfq1ib5e4kmwjehxtpjotfdszl.tar
Reusing existing connection to northwestern.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Locat

## Play the game

Once you see "Setup complete" in the box above, click the arrow in the box below to start running the game. It will take a couple of seconds for the game to generate the first prompt.

**Tips**


*   The game works best if you give it simple commmands, i.e. "go west" or "open box"
*   If you're not sure what to do, you can say "Look around" or enter nothing at all and the game will continue descriving your current location.
* Use to "go" command to move. You can say use a direction ("go west") or a landmark ("go to tree")


In [ ]:
#Click the number or arrow to the left to run


history="You are standing in a field. A unicorn grazes peacefully nearby." 

def isContext(text): #is a iece of textthis a context change? (i.e. moving to a new room)
  if "you are" in text or "we are" in text or "this is" in text or '''you're''' in text: #context change
    return True
  else:
    return False

def isTakeLoop(playerAction,result): #lets us detect a loop condition where the game just replies "taken" or "done" to everything
  if "done" in result.lower() or ( "take" in result.lower() and "take" not in playerAction.lower()):
    return True
  else:
    return False
  
def isInvalidMove(playerAction,result): #gpt is trying to move the player when they didn't specify they wanted to move
  if ("go ") in playerAction:
    return False
  else:
    if isContext(result):
      return True
    else:
      return False
    
   

#This text defines where your character starts! You can make it anything!

STRICT_MODE=True #Change to True to make the game a bit more coherent, but possibly a bit less interesting

locContext=""
alreadyDone=""
prompt=""
action=""
firstRun=True #for the first response (generating the initial setting) we want to impose less strict restrictions on output
while True:
  newprompt="no response"
  gpo=gpt2.generate(sess, temperature=0.1,prefix=history,run_name='run1',length=100,return_as_list=True,nsamples=3,batch_size=3,top_p=0.99)
  for candidate in gpo:
    goodCandidate=False
    result=candidate
    splitup=result.split("\n")
    newprompt=""
    hasContext=False
    for item in splitup:
      words=item.split(" ")
      if (item not in alreadyDone or len(words) <= 2) and item not in newprompt and (isTakeLoop(action,item)==False) and (isInvalidMove(action,item)==False or firstRun):#avoid repeating things we've said in this or previous responses. If a response is very short (i.e. "taken" when you pick up an item) it's ok to repeat.
        if ("." in item or "?" in item or "!" in item) and (hasContext==False or  isContext(item)==False) : #If it's a next player action, then stop, otherwise keep going
          newprompt=newprompt+item
          goodCandidate=True
          if isContext(item):
            hasContext=True
        else:
          if goodCandidate and STRICT_MODE: #This prevents GPT from taking actions on our behalf. If strict mode is on actions are never taken, if it is off they are taklen but not shown to us. Strict mode on can make the game more playable at the expense of less interesting descriptions 
            break
    if goodCandidate:
      break
  if len(newprompt) > 3: #this will be blank if GPT couldn't come up with anything
    if isContext(newprompt): #This is updating the location context
      locContext=newprompt
    print(newprompt.replace(".","\n").upper())
    prompt=newprompt
    action=input()
    firstRun=False
    history=locContext+"\n"+prompt+"\n"+action
    alreadyDone=alreadyDone+prompt+"\n"+action
  else:
    print("I don't know how to do that".upper())
    newaction=input()
    history=locContext+"\n"+prompt+"\n"+newaction
    alreadyDone=alreadyDone+"\n"+newaction
    
  

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
YOU ARE STANDING IN A FIELD
 A UNICORN GRAZES PEACEFULLY NEARBY

go to the moon
YOU ARE ON THE MOON
 A PATHETICALLY BABBLING TROLL IS STANDING IN THE CORNER, EYEING YOU CLOSELY

talk to troll
 THE TROLL ISN'T MUCH OF A CONVERSATIONALIST
THE TROLL, ANGERED AND HUMILIATED, RECOVERS HIS WEAPON
 HE APPEARS TO HAVE AN AXE TO GRIND WITH YOU
THE TROLL, DISARMED, COWERS IN TERROR, PLEADING FOR HIS LIFE IN THE GUTTURAL TONGUE OF THE TROLLS
THE TROLL STIRS, QUICKLY RESUMING A FIGHTING STANCE

what weapons do i have
WHAT WEAPONS DO I HAVE?
check inventory
I DON'T KNOW HOW TO DO THAT
punch troll
YOU HAVE PUNCH CARDS, NO DOUBT JESTING TO THEMSELVES

escape from the troll
THIS ROOM IS A PRIVATE ROOM, WHICH MIGHT BE ACCESSIBLE FROM ANY CORNER OF THE ROOM
 PERSONS WHO ARE NOT CRIMINALS MAY NOT BE ABLE TO ENTER IT, BUT THERE IS A WAY THROUGH WHICH YOU MAY GET INTO IT

get into it
 YOU SPLASH AROUND IN THE ROOM

# LICENSE

MIT License

Copyright (c) 2019 Nathan Whitmore, Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.